In [18]:
import redis
import time
import pickle

In [179]:
class RedisLog():
    def __init__(self, sname = 'RedisLog', fname = 'NA', status = 1, error = None, uuid = 0):
        self.service_name = sname
        self.func_name = fname
        self.status = status
        self.error = error
        self.uuid = uuid
        self.timestamp = int(time.time())
    def print(self):
        print("Service Name:%s"%self.service_name)
        print("Function Name:%s"%self.func_name)
        print("Status:%s"%self.status)
        print("Error:%s"%self.error)
        print("UUID:%s"%self.uuid)
        print("Timestamp:%s"%self.timestamp)
            
class Redis():
    def __init__(self, host, port, password):
        try:
            self.redis = redis.StrictRedis(host = host,
                                    port = port,
                                    password = password)
        except Exception as e:
            #redis can not be connected
            self.redis = None
            #user should check if redis is none or not before proceeding
            pass
    def serialize(self, objs):
        """
        objs: list of python objects
        return: list of picked objects, [] if failed
        """
        try:
            pobjs=[]
            for o in objs:
                pobjs.append(pickle.dumps(o))
            return pobjs
        except Exception as e:
            print (e)
            return []
    def set_expire(self, key, ts):
        """
        key: service name
        ts: time in seconds
        return: -1 if fail
        """
        try:
            self.redis.pexpire(key,ts*1000)
        except Exception as e:
            print (e)
            return -1
    def get_ttl(self, key):
        """
        key: service name
        return: time (seconds) before expire, -1 if fail
        """
        try:
            t = self.redis.pttl(key)
            return t/1000
        except Exception as e:
            print(e)
            return -1
    def put(self, key, values):
        """
        key: service name
        values: list of logs or a single log
        return: number of logs inserted, 0 if nothing inserted        
        """
        if isinstance(values, list):
            if(len(values) ==0):
                return 0
        else:
            if values:
                values = [values]
            else:
                # values is none
                return 0
        try:
            #push all values into redis' list tail
            #serialize first
            vobjs = self.serialize(values)
            #push all objects to redis 
            if self.redis:
                self.redis.rpush(key,*vobjs)
                return len(vobjs)
            else:
                return 0
        except Exception as e:
            #in case of expection, push a simple error log into redis
            print (e)
            rlog = RedisLog(fname = 'rpush', status = 0, error = e)
            rlog_obj = self.serialize([rlog])
            try:
                self.redis.rpush('RedisLog',rlog_obj)
            except Exception as e:
                #redis failed with best try
                print (e)
                return 0
    def get(self, key, num=None):
        """
        key: service name
        num: number of logs to get
        return: list of RedisLog or [] if none found
        """
        #get latest num logs from service key
        Logs = []
        try:
            if num != None and num >0:
                objs = self.redis.lrange(key, -num, -1)
            else:
                objs = self.redis.lrange(key,0,-1)
                #print("objs:",objs)
            for o in objs:
                Logs.append(pickle.loads(o))
            return Logs
        except Exception as e:
            print (e)
            return []

In [207]:
#For local test on mac
#redis-server /usr/local/etc/redis.conf
#redis-cli shutdown
#redis-cli  -h localhost -p 6379 -a sunmiai2020 lrange  -3 -1 humanalarm
import os
class Test_Redis():
    def __init__(self):
        self.r = None
    def test_connect(self):
        #setting up Redis connection
        host = os.getenv('RedisHost', default = 'localhost')
        port = os.getenv('RedisPort', default = '6379')
        password = os.getenv('RedisPass', default = 'idontknow')
        r = Redis(host, port, password)
        self.r = r
    def test_put_onelog(self):
        #Test inserting a single log
        rlog0 = RedisLog(sname = 'humanalarm', 
                         fname = 'detection',
                         status = 0,
                         error = 'single log error',
                         uuid = 1000)
        key = 'humanalarm'
        l = self.r.put(key, rlog0)
        assert(l == 1)
    def test_put_logs(self):
        #Test inserting a list of logs
        RLogs=[]
        rlog1 = RedisLog(sname = 'humanalarm', 
                         fname = 'detection',
                         status = 1,
                         error = '',
                         uuid = 1000133)
        RLogs.append(rlog1)
        time.sleep(1)

        rlog2 = RedisLog(sname = 'humanalarm', 
                         fname = 'detection',
                         status = 0,
                         error = 'some expection',
                         uuid = 1000133)
        RLogs.append(rlog2)
        #Add logs into redis
        key = 'humanalarm'
        l = self.r.put(key, RLogs)
        assert(l==len(RLogs)) 
    def test_get_latest(self):
        #Test retrieving latest #num logs
        key = 'humanalarm'
        num = 3 # retrieve latest 3 logs for service 'humanalarm'
        logs = self.r.get(key,num)
        for log in logs:
            log.print()
    def test_get_all(self):
        #Test retrieving all logs for service 
        key = 'humanalarm'
        logs = self.r.get(key) 
        if logs:
            for log in logs:
                log.print()
        print('Total Logs:%d'%len(logs))
    def test_set_expire(self):
        #Test setting expiration time for a service
        key = 'humanalarm'
        ts = 30 # set it expire after 60 seconds
        self.r.set_expire(key,ts)
        print('TTL for service:%s = %f'%(key,ts))
        time.sleep(10)
        ttl = self.r.get_ttl(key)
        print('TTL for service:%s = %f'%(key,ttl))
        time.sleep(20)
        ttl = self.r.get_ttl(key)
        print('TTL for service:%s = %f'%(key,ttl))
        time.sleep(5)
        ttl = self.r.get_ttl(key)
        print('TTL for service:%s = %f'%(key,ttl))
        time.sleep(5)
        ttl = self.r.get_ttl(key)
        print('TTL for service:%s = %f'%(key,ttl))

In [208]:
test = Test_Redis()

In [209]:
test.test_connect()

In [210]:
test.test_put_onelog()

In [211]:
test.test_put_logs()

In [212]:
test.test_get_latest()

Service Name:humanalarm
Function Name:detection
Status:0
Error:single log error
UUID:1000
Timestamp:1589321124
Service Name:humanalarm
Function Name:detection
Status:1
Error:
UUID:1000133
Timestamp:1589321124
Service Name:humanalarm
Function Name:detection
Status:0
Error:some expection
UUID:1000133
Timestamp:1589321125


In [213]:
test.test_get_all()

Service Name:humanalarm
Function Name:detection
Status:0
Error:single log error
UUID:1000
Timestamp:1589321124
Service Name:humanalarm
Function Name:detection
Status:1
Error:
UUID:1000133
Timestamp:1589321124
Service Name:humanalarm
Function Name:detection
Status:0
Error:some expection
UUID:1000133
Timestamp:1589321125
Total Logs:3


In [214]:
test.test_set_expire()

TTL for service:humanalarm = 30.000000
TTL for service:humanalarm = 19.999000
TTL for service:humanalarm = -0.002000
TTL for service:humanalarm = -0.002000
TTL for service:humanalarm = -0.002000


In [215]:
test.test_get_latest()

In [216]:
test.test_get_latest()